# Aufgabe 3: Gaußfilter
Anstatt wie in der vorigen Aufgabe an einer ausgewählten Frequenz hart abzuschneiden, kann mittels eines Faktors
\begin{align}
 h(u,v) = \mathrm{e}^{-\frac{D^2(u,v)}{2\sigma^2}},\qquad h \in [0;1]
\end{align}
auch ein weicher Übergang erzeugt werden.

Verwenden Sie eine Gauß-Glocke für die Hoch- und Tiefpassfilterung und berechnen Sie den Faktor $h$ in Abhängigkeit von der Entfernung $D(u,v)$ zum Ursprung!
Führen Sie die Transformationen ansonsten wie in Teilaufgabe 2 durch! Die Varianz $\sigma$ der Gauß-Glocke ist mit dem dort gegebenen Radius gleichzusetzen.
Was ändert sich in den Ausgabebildern im Vergleich zu Teilaufgabe 2?

## 0. Pfade, Pakete etc.

In [ ]:
import glob
import imageio
import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
image_filter = '../Bilder/*.jpg'

## 1. Definition des Filters

Übernehmen Sie hier zu Vergleichszwecken Ihre Definition des idealen Hoch- und Tiefpass-Filters aus der vorigen Aufgabe:

In [ ]:
D0 = 20
lowpass_filter = lambda D: 1 if D<=D0 else 0
highpass_filter = lambda D: 1 if D>D0 else 0

Definieren Sie den Gauß-Filter als Funktion des Radius $D$ und der Konstante $\sigma$:

In [ ]:
sigma = 20
gauss_lowpass = lambda D: np.exp(-D**2/(2*sigma*sigma))
gauss_highpass = lambda D: 1-np.exp(-D**2/(2*sigma*sigma))

## 2. Laden und Normalisieren des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

In [ ]:
image = image.astype(np.float32)
image -= image.min()
image /= image.max()

## 3. Berechnung der Fouriertransformation
Setzen Sie hier ihre Lösung aus der vorigen Aufgabe ein:

In [ ]:
image_transformed = np.fft.fftshift(np.fft.fft2(image))

## 4. Filterung

Setzen Sie hier ihre Lösung `ex3_filter_spectrum` aus der vorigen Aufgabe ein:

In [ ]:
def ex3_filter_spectrum(spectrum, radial_filter):
    H = np.zeros(np.shape(image))
    y,x = np.shape(image)
    for v in range(y):
        for u in range(x):
            D = np.sqrt((v-y/2)**2 + (u-x/2)**2)
            H[v,u]=radial_filter(D)

    spectrum = spectrum * H
    return spectrum

Das transformierte Bild (Spektrum) wird nun gefiltert:

In [ ]:
image_transformed_lowpass = ex3_filter_spectrum(image_transformed, lowpass_filter)
image_transformed_highpass = ex3_filter_spectrum(image_transformed, highpass_filter)
image_transformed_gauss_lowpass = ex3_filter_spectrum(image_transformed, gauss_lowpass)
image_transformed_gauss_highpass = ex3_filter_spectrum(image_transformed, gauss_highpass)

## 5. Inverse Filterung
Das veränderte Spektrum soll nun in den Ortsbereich zurücktransformiert werden. Verwenden Sie dazu die entsprechenden Funktionen des Paketes `numpy.fft`.

In [ ]:
image_filter_lowpass = np.abs(np.fft.ifft2(np.fft.ifftshift(image_transformed_lowpass)))
image_filter_highpass = np.abs(np.fft.ifft2(np.fft.ifftshift(image_transformed_highpass)))
image_filter_gauss_lowpass = np.abs(np.fft.ifft2(np.fft.ifftshift(image_transformed_gauss_lowpass)))
image_filter_gauss_highpass = np.abs(np.fft.ifft2(np.fft.ifftshift(image_transformed_gauss_highpass)))

Vergleichen Sie nun die Ergebnisses des idealen Hoch- und Tiefpassfilters mit den Gauß-Filtern:

In [ ]:
plt.figure('Convolution: image comparison', figsize=(12, 6))
plt.subplot(1,4,1, title='Lowpassed Image')
plt.imshow(image_filter_lowpass, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,4,2, title='Highpassed Image')
plt.imshow(image_filter_highpass, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,4,3, title='Gauss Lowpassed Image')
plt.imshow(image_filter_gauss_lowpass, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,4,4, title='Gauss Highpassed Image')
plt.imshow(image_filter_gauss_highpass, cmap='gray', vmin=0, vmax=1)
plt.show()

### Welche Unterschiede lassen sich im Vergleich zu Aufgabe 2 feststellen?

Die Bilder sind smoother